In [1]:
import sys, time, wmi, psutil
SYSTEM_INFO = wmi.WMI().Win32_OperatingSystem()[0]
"system: {0}, {1}, {2}".format(SYSTEM_INFO.Caption, SYSTEM_INFO.BuildNumber, SYSTEM_INFO.OSArchitecture) 
"memory: {}G".format(round(psutil.virtual_memory().total / 1024**3, 2))
"cpu: {}".format(psutil.cpu_count())
"python: {}".format(sys.version)
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

'system: Microsoft Windows 10 教育版, 18363, 64 位'

'memory: 15.86G'

'cpu: 4'

'python: 3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]'

'2020-10-09 10:04:54'

- **@author**: run_walker
- **@references**:
    1. [Wikipedia -> Cycle Detection](https://en.wikipedia.org/wiki/Cycle_detection) 还有一些应用及其他算法
    2. [Siafoo -> Cycle Detection](http://www.siafoo.net/browse?keyword_id=711)

# 概念
[Wikipedia -> Cycle detection](https://en.wikipedia.org/wiki/Cycle_detection)
1. In computer science, **cycle detection** or **cycle finding** is the algorithmic problem of finding a cycle in a sequence of iterated function values.           
2. For any function $f$ that maps a **finite set** $S$ to itself, and any initial value $x_0$ in S, the sequence of iterated function values
$x_{0},~x_{1}=f(x_{0}),~x_{2}=f(x_{1}),~...,~x_{i}=f(x_{i-1}),~...~$must eventually use the same value twice: there must be some pair of distinct indices $i$ and $j$ such that $x_i = x_j$. Once this happens, the sequence must continue periodically, by repeating the same sequence of values from $x_i$ to $x_{j − 1}$. Cycle detection is the problem of finding $i$ and $j$, given $f$ and $x_0$.
3. Several algorithms for finding cycles quickly and with little memory are known. **Floyd's tortoise and the hare algorithm** moves two pointers at different speeds through the sequence of values until they both point to equal values. Alternatively, **Brent's algorithm** is based on the idea of **exponential search**. Both Floyd's and Brent's algorithms use only a constant number of memory cells, and take a number of function evaluations that is proportional to the distance from the start of the sequence to the first repetition. *Several other algorithms trade off larger amounts of memory for fewer function evaluations*.

# 问题
如何检测一个链表是否有环（循环节），如果有，如何确定环的起点和环的长度。<font color="red">注意：不能存储经过的每一个点，要求常数空间复杂度。</font>

# 算法

## Floyd's Tortoise and Hare Algorithm
**Floyd判圈算法**（*Floyd's cycle-finding algorithm*），也被称为**龟兔赛跑算法**（由Aesop's fables中龟兔赛跑故事得名）等等。

### 历史
The algorithm is named after **Robert W. Floyd**, who was credited with its invention by **Donald Knuth**. However, the algorithm does not appear in Floyd's published work, and this may be a misattribution: Floyd describes algorithms for listing all simple cycles in a directed graph in a 1967 paper, but this paper does not describe the cycle-finding problem in functional graphs that is the subject of this article. In fact, Knuth's statement (in 1969), attributing it to Floyd, without citation, is the first known appearance in print, and it thus may be a folk theorem（无名氏定理）, not attributable to a single individual.

### 算法流程
令乌龟、兔子为指针，开始时均指向起点位置，乌龟和兔子每次向后分别移动一个和两个节点。
1. **判断是否有环**：如果两者在某一位置相遇（开始时位于同一位置不算），则说明有环，若兔子到达链表尾部，则说明无环。
2. **寻找环的起点**：令两个指针分别从起点和相遇位置出发，同时向后一步一步移动，则两指针首次相遇位置即为环的入口。
3. **计算环的长度**：令一个指针从相遇位置出发，一步一步向后移动并记录步数，再次回到原位置时步数即环的长度。

### 有效性证明
**Proof of 1:**
1. 若存在环，则进入环中后无法出来，所以非环节点均位于环之前。假设环之前有$m$个节点，序号依次从$0$记到$m-1$，环内有$n$个节点，序号依次从$m$记到$m+n-1$。特别地，如果环之前没有节点，则乌龟和兔子从起始点出发后，乌龟（经过一圈）回到该位置与兔子（经过两圈）回到该位置相遇。
2. 则迭代函数的值序列$x_i$为所在节点的序号，$x_0=0,...,x_{m-1}=m-1,x_m=m,...,x_{m+n-1}=m+n-1$，然后继续从$m$到$m+n-1$循环下去。也即是说，$\forall i\geq m,~j\in\mathbb{Z}_+$，有$x_i=x_{i+jn}$。
3. 兔子速度为乌龟2倍，所以当乌龟位于$x_i$时，兔子应当位于$x_{2i}$，乌龟第一次进入环后，在环中依次经过$x_m,...,x_{m+n-1}$，兔子所在位置和乌龟所在位置下标差也依次为$m,m+1,...,m+n-1$。
4. 记$m=a*n+b$，其中$a,b$均为非负整数，且$0\leq b<n$。
    1. 若$b=0$，则$m=a*n$，
    2. 若$b>0$，则$m=an+b<(a+1)n<(a+1)n+b=m+n$
5. 4中分析表明乌龟在环中移动时，兔子与其位置的下标差中一定存在$n$的倍数，也即兔子超过乌龟至少一圈后与其相遇在环中某一位置，记为序号$m+l$，其中$0\leq l\leq n-1$。

**Proof of 2:**
从1的证明中可知，$m+l=kn$。指针1从起点向后运动$m$步后抵达环的入口$m$处，在这段路程中指针1均位于环之外，不可能和指针2相遇，而指针2从位置$m+l$向右移动$m$步后抵达位置$x_{2m+l}=x_{m+kn}=x_m=m$，也即在环上移动了$k$圈后同样抵达环的入口$m$处，也即和指针1相遇在环的入口处。

### 复杂度分析
常数空间复杂度，线性时间复杂度。有环时最差的情况下乌龟要第二次移动到环的入口，也即$O(m+n)$，无环时兔子要遍历完整个链表，同样为$O(m+n),~n=0$。

### 可视化演示

In [4]:
from IPython.display import IFrame
IFrame(src="https://visualgo.net/zh/cyclefinding", width="100%", height=700)

### python实现
要先判断是否存在环，而确定环的起点和长度这两步并无先后顺序。

In [5]:
def floyd(f, x0):
    """
    这里我们假定传进来的f将有限集映至自身，是一定存在环的；
    而实际应用中更多的情形是不确定是否有环的，要修改代码增加可能的终止条件，
    例如链表判圈的终止条件就是抵达某个节点后它不再指向下一个节点。
    
    Parameters
    ----------
    f
    x0

    Returns
    -------
    mu: start pos
    lam: length of cycle

    """
    tortoise = f(x0) 
    hare = f(tortoise)
    while tortoise != hare:  # 事实上这里应当是：(没达到终止条件) or tortoise != hare
        tortoise, hare = f(tortoise), f(f(hare))  # 或者加在这里，每对hare迭代求一次f值，就判断是否达到终止条件
    
    # 如果是通过终止条件停止的循环，说明没有环
    
    # Find the position μ of first repetition.    
    mu = 0
    tortoise = x0
    while tortoise != hare:
        tortoise, hare = f(tortoise), f(hare)   
        mu += 1
 
    # Find the length of the shortest cycle starting from x_μ
    lam = 1
    hare = f(tortoise)
    while hare != tortoise:
        hare = f(hare)
        lam += 1
 
    return mu, lam

In [8]:
f = lambda x: (3 * x**2 + 7 * x + 5) % 97
nums = [62]
for _ in range(17):
    nums.append(f(nums[-1]))

nums

[62, 40, 41, 0, 5, 18, 36, 71, 8, 59, 94, 11, 57, 63, 34, 25, 18, 36]

In [9]:
floyd(f, 62)

(5, 11)

## Brent's Algorithm
*Brent's Cycle Detection Algorithm(The Teleporting Turtle)*

teleporting （通常见于科幻作品）（被）远距离传送

### 历史
In 1980, Richard P. Brent invented an algorithm that not only worked in linear time, but required less stepping than Floyd's Tortoise and Hare algorithm (however it is slightly more complex).

### 算法流程
1. 令乌龟、兔子为指针，开始时均指向起点位置。
2. 兔子每轮向后移动一步，然后判断是否和乌龟位置相同：如果不同则记录步数并继续向后移动，如果相同则表明有环并且环的长度为记录的步数。
3. 在兔子移动的过程中，每经过$2$的幂次方次移动后，将乌龟传送到兔子所在位置，并且每次幂都将翻倍。详细来讲：当兔子首次移动$2^0=1$步后，将乌龟移至兔子位置；当兔子再次移动$2^1=2$步后，将乌龟移至兔子位置；当兔子再次移动$2^2=4$步后，将乌龟移至兔子位置，以此类推。注意每次传送乌龟后，将步数记录归$0$。
4. 如果兔子抵达终点，则表明无环。
5. 若有环，则同时已知环的长度$n$，令兔子领先乌龟$n$步后，同时向后一步一步移动，第一次相遇的地方即为环的起点。

<img src="imgs/Brent's_Algorithm.png" style="width:250px">

### 有效性证明
如果存在环，当兔子率先进入环内后，环外的乌龟会在$2$的幂次方时传送至兔子所在位置，随着$2$的幂次方的翻倍，必然会在某一次超过环的长度，在这一段中兔子将再次移动到乌龟所在位置，发生相遇。

### 复杂度分析
Like *The Tortoise and Hare Algorithm*, this is a pointer algorithm that uses $O(m+n)$ tests and function evaluations and $O(1)$ storage space. **It is not difficult to show that the number of function evaluations can never be higher than for Floyd's algorithm**（也即是说，Floyd算法是Brent算法的最差表现，**待证明**）. Brent claims that, on average, his cycle finding algorithm runs around 36% more quickly than Floyd's and that it speeds up the Pollard rho algorithm by around 24%. 

<div class="alert alert-block alert-info">
    <i class="fa fa-list-alt" aria-hidden="true"><b> Todo:</b></i>
    待证明：Floyd算法是Brent算法的最差表现。
</div>

It has two advantages compared to *The Tortoise and Hare Algorithm*: 
1. it finds the correct length λ of the cycle directly, rather than needing to search for it in a subsequent stage.
2. its steps involve only one evaluation of f rather than three.

### python实现

In [12]:
def brent(f, x0):
    """
    这里我们假定传进来的f将有限集映至自身，是一定存在环的；
    而实际应用中更多的情形是不确定是否有环的，要修改代码增加可能的终止条件，
    例如链表判圈的终止条件就是抵达某个节点后它不再指向下一个节点。
    
    Parameters
    ----------
    f
    x0

    Returns
    -------
    m: start pos
    n: length of cycle

    """
    # main phase: search successive powers of two
    tortoise = x0
    hare = f(x0)
    power = n = 1  # 2^0
    while tortoise != hare:  # 事实上这里应当是：(没达到终止条件) or tortoise != hare
        if power == n:  # time to start a new power of two
            tortoise = hare
            power *= 2
            n = 0
        hare = f(hare)
        n += 1
    
    # 如果是通过终止条件停止的循环，说明没有环

    # Find the position μ of first repetition. 
    tortoise = hare = x0
    for i in range(n):
        hare = f(hare)
    #
    m = 0
    while tortoise != hare:
        tortoise, hare = f(tortoise), f(hare)
        m += 1

    return m, n

In [13]:
f = lambda x: (3 * x**2 + 7 * x + 5) % 97
nums = [62]
for _ in range(17):
    nums.append(f(nums[-1]))

nums

[62, 40, 41, 0, 5, 18, 36, 71, 8, 59, 94, 11, 57, 63, 34, 25, 18, 36]

In [14]:
brent(f, 62) 

(5, 11)

# 例题

## 快乐数
- question: [LeetCode > 202. 快乐数](https://leetcode-cn.com/problems/happy-number/)
- solution: [LeetCode > 判圈算法习题合集，一文打尽！](https://leetcode-cn.com/problems/linked-list-cycle-ii/solution/pan-quan-suan-fa-xi-ti-he-ji-yi-wen-da-jin-by-a_ru/)

## 环形链表
- question: [LeetCode > 141. 环形链表](https://leetcode-cn.com/problems/linked-list-cycle/)
- solution: [LeetCode > 判圈算法习题合集，一文打尽！](https://leetcode-cn.com/problems/linked-list-cycle-ii/solution/pan-quan-suan-fa-xi-ti-he-ji-yi-wen-da-jin-by-a_ru/)

## 环形链表 II
- question: [LeetCode > 142. 环形链表 II](https://leetcode-cn.com/problems/linked-list-cycle-ii/)
- solution: [LeetCode > 判圈算法习题合集，一文打尽！](https://leetcode-cn.com/problems/linked-list-cycle-ii/solution/pan-quan-suan-fa-xi-ti-he-ji-yi-wen-da-jin-by-a_ru/)

## 寻找重复数
- question: [LeetCode > 287. 寻找重复数](https://leetcode-cn.com/problems/find-the-duplicate-number)
- solution: [LeetCode > 判圈算法习题合集，一文打尽！](https://leetcode-cn.com/problems/linked-list-cycle-ii/solution/pan-quan-suan-fa-xi-ti-he-ji-yi-wen-da-jin-by-a_ru/)